In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
base_path = Path().resolve().parent
%cd {base_path}

/home/optima/mhaderer/AILS-MICCAI-UWF4DR-Challenge


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# setup
#!apt-get update
#!pip install python-dotenv
#!pip install loguru
#!pip install gdown
#!pip install typer
#!pip install imbalanced-learn

In [4]:
# load data and unzip data

#!python tools/download_data_and_chkpts.py

In [5]:
# imports

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import wandb

from sklearn.metrics import roc_auc_score, average_precision_score

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import time

# data
from ails_miccai_uwf4dr_challenge.dataset_strategy import CustomDataset, DatasetStrategy, CombinedDatasetStrategy, \
    Task1Strategy, Task2Strategy, Task3Strategy, DatasetBuilder

# augmentation
from ails_miccai_uwf4dr_challenge.preprocess_augmentations import ResidualGaussBlur, MultiplyMask

# trainer
from ails_miccai_uwf4dr_challenge.models.trainer import DefaultMetricsEvaluationStrategy, Metric, MetricCalculatedHook, \
    NumBatches, Trainer, EpochTrainingStrategy, EpochValidationStrategy, DefaultEpochTrainingStrategy, \
    DefaultBatchTrainingStrategy, TrainingContext, PersistBestModelOnEpochEndHook
from ails_miccai_uwf4dr_challenge.models.metrics import sensitivity_score, specificity_score
from ails_miccai_uwf4dr_challenge.config import Config

# models
from ails_miccai_uwf4dr_challenge.models.architectures.task1_automorph_plain import AutoMorphModel
from ails_miccai_uwf4dr_challenge.models.architectures.task1_efficientnet_plain import Task1EfficientNetB4
from ails_miccai_uwf4dr_challenge.models.architectures.task2_efficientnetb0_plain import Task2EfficientNetB0 
from ails_miccai_uwf4dr_challenge.models.architectures.task1_convnext import Task1ConvNeXt 
from ails_miccai_uwf4dr_challenge.models.architectures.dinov2 import DinoV2Classifier, ModelSize

from ails_miccai_uwf4dr_challenge.config import WANDB_API_KEY, PROJ_ROOT

wandb.login(key=WANDB_API_KEY)

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations
wandb: Currently logged in as: mmarinschek (miccai-challenge-2024). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/optima/mhaderer/.netrc


True

In [6]:
# select device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))

Device: cuda


## Train some model

In [7]:
def train_model(cfg=None):

    WANDB_HTTP_TIMEOUT=300
    WANDB_INIT_TIMEOUT =600
    WANDB_DEBUG=True

    wandb.init(project="task2", config=cfg)
    cfg = wandb.config

    transforms_train = A.Compose([
        A.Resize(800, 1016, p=1),
        MultiplyMask(p=1),
        ResidualGaussBlur(p=cfg.p_gaussblur),
        A.Equalize(p=cfg.p_equalize),
        A.CLAHE(clip_limit=5., p=cfg.p_clahe),
        A.HorizontalFlip(p=cfg.p_horizontalflip),
        A.Affine(rotate=cfg.rotation, rotate_method='ellipse', p=cfg.p_affine),
        A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
        #A.Resize(770, 1022, p=1), # comment whenever not using DinoV2
        ToTensorV2(p=1)
    ])

    transforms_val = A.Compose([
            A.Resize(800, 1016, p=1),
            MultiplyMask(p=1),
            A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
            #A.Resize(770, 1022, p=1), # comment whenever not using DinoV2
            ToTensorV2(p=1)
        ])

    dataset_strategy = CombinedDatasetStrategy()
    task_strategy = Task2Strategy()

    builder = DatasetBuilder(dataset_strategy, task_strategy, split_ratio=0.8)
    train_data, val_data = builder.build()

    train_dataset = CustomDataset(train_data, transform=transforms_train)
    val_dataset = CustomDataset(val_data, transform=transforms_val)

    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=cfg.batch_size, shuffle=False)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu" if torch.backends.mps.is_available() else "mps")
    print(f"Using device: {device}")

    assert cfg.model is not None, "Model type must be specified in the config"

    if cfg.model == "Task2EfficientNetB0":
        model = Task2EfficientNetB0(num_classes=1)
    elif cfg.model == "AutoMorphModel":
        model = AutoMorphModel(enc_frozen=True)
    elif cfg.model == "DinoV2Classifier":
        model = DinoV2Classifier(ModelSize.SMALL)
    else:
        raise ValueError(f"Model type {cfg.model} not recognized")

    model.to(device)

    metrics = [
        Metric('auroc', roc_auc_score),
        Metric('auprc', average_precision_score),
        Metric('accuracy', lambda y_true, y_pred: (y_pred.round() == y_true).mean()),
        Metric('sensitivity', sensitivity_score),
        Metric('specificity', specificity_score)
    ]

    class WandbLoggingHook(MetricCalculatedHook):
        def on_metric_calculated(self, training_context: TrainingContext, metric: Metric, result, last_metric_for_epoch: bool):
            import wandb
            wandb.log(data={metric.name: result}, commit=last_metric_for_epoch)

    metrics_eval_strategy = DefaultMetricsEvaluationStrategy(metrics).register_metric_calculated_hook(WandbLoggingHook())

    def combined_losses(pred, target):
        bce = F.binary_cross_entropy_with_logits(pred, target) * cfg.loss_weight
        smooth_l1 = F.smooth_l1_loss(pred, target) * (1 - cfg.loss_weight)
        return bce + smooth_l1

    criterion = combined_losses
    optimizer = optim.AdamW(model.parameters(), lr=cfg.lr)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=cfg.lr_schedule_factor, patience=cfg.lr_schedule_patience, verbose=True)

    trainer = Trainer(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, device, 
                        metrics_eval_strategy=metrics_eval_strategy, resampling_strategy=cfg.resampling_strategy)

    # build a file name for the model weights containing current timestamp and the model class
    wandb_run_name = wandb.run.name

    training_timestamp = time.strftime("%Y-%m-%d_%H-%M")
    persist_model_hook = PersistBestModelOnEpochEndHook(f"models/{wandb_run_name}_{training_timestamp}.pth")
    trainer.add_epoch_end_hook(persist_model_hook) # TODO uncomment this line to save the best model

    #print("First train 2 epochs 2 batches to check if everything works - you can comment these two lines after the code has stabilized...")
    #trainer.train(num_epochs=2, num_batches=NumBatches.TWO_FOR_INITIAL_TESTING)
    
    print("Now train train train")
    trainer.train(num_epochs=cfg.epochs)
    wandb.finish()
    print("Finished training")

In [8]:
cfg = Config(
    batch_size=32,
    epochs=25,
    lr=0.001,
    lr_schedule_factor=0.1,
    lr_schedule_patience=5,
    p_gaussblur=0,
    p_equalize=0,
    p_clahe=0.3,
    p_horizontalflip=0.3,
    rotation=15,
    p_affine=0.3,
    loss_weight=0.5,
    resampling_strategy='undersampling',
    model="Task2EfficientNetB0",
)

In [9]:
#train_model(cfg)

## SWEEP

In [11]:

# Define the sweep configuration
sweep_config = {
    "method": "random",  # or "grid", or "bayes"
    "parameters": {
        "model": {
            "values": ["Task2EfficientNetB0"]
        },
        "lr": {
            "values": [1e-3]
        },
        "epochs": {
            "values": [30]
        },
        "batch_size": {
            "values": [16]
        },
        "p_gaussblur": {
            "values": [0, 0.3]
        },
        "p_equalize": {
            "values": [0]
        },
        "p_clahe": {
            "values": [0.3, 0.5]
        },
        "p_horizontalflip": {
            "values": [0.5]
        },
        "rotation": {
            "values": [10]
        },
        "p_affine": {
            "values": [0.3]
        },
        "loss_weight": {
            "values": [0.5]
        },
        "lr_schedule_factor": {
            "values": [0.1]
        },
        "lr_schedule_patience": {
            "values": [5]
        },
        "resampling_strategy": {
            "values": ['oversampling', 'undersampling']
        }
    }
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="task2")

# Start the sweep
wandb.agent(sweep_id, function=train_model)

INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=None, count=None


Create sweep with ID: eo1h97d3
Sweep URL: https://wandb.ai/miccai-challenge-2024/task2/sweeps/eo1h97d3


wandb: Agent Starting Run: l1cer1et with config:
wandb: 	batch_size: 16
wandb: 	epochs: 30
wandb: 	loss_weight: 0.5
wandb: 	lr: 0.001
wandb: 	lr_schedule_factor: 0.1
wandb: 	lr_schedule_patience: 5
wandb: 	model: Task2EfficientNetB0
wandb: 	p_affine: 0.3
wandb: 	p_clahe: 0.3
wandb: 	p_equalize: 0
wandb: 	p_gaussblur: 0.3
wandb: 	p_horizontalflip: 0.5
wandb: 	resampling_strategy: oversampling
wandb: 	rotation: 10
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Using device: cuda
Loaded pretrained weights for efficientnet-b0
Now train train train


Epoch 1/30 - Avg val Loss: 0.334822: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.46s/it]


...

Epoch 10/30 - Avg val Loss: 0.291955: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.46s/it]


New best model found at epoch 10 with validation loss: 0.2920. Model saved to models/firm-sweep-28_2024-07-24_09-39.pth


Epoch 11/30 - Avg val Loss: 0.263183: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.61s/it]


New best model found at epoch 11 with validation loss: 0.2632. Model saved to models/firm-sweep-28_2024-07-24_09-39.pth


Epoch 15/30 - Avg train Loss: 0.243258:  33%|██████████████████████████████████████████▎                                                                                    | 5/15 [00:09<00:17,  1.77s/it]wandb: Ctrl + C detected. Stopping sweep.


'''
class ResamplingStrategy(ABC):
    @abstractmethod
    def apply(self, dataloader, epoch):
        pass


    def _calculate_weights(self, dataloader):
        dataset = dataloader.dataset
        labels = [item[1] for item in dataset]
        
        class_counts = Counter(labels)
        
        weights = [1.0 / class_counts[label] for label in labels]
        return weights
    

class DefaultResamplingStrategy(ResamplingStrategy):
    def apply(self, dataloader, epoch):
        # no resampling is applied by default
        return dataloader
    

class OversamplingResamplingStrategy(ResamplingStrategy):
    def apply(self, dataloader, epoch):

        weights = self._calculate_weights(dataloader)

        sampler = WeightedRandomSampler(weights, len(weights), replacement=True)

        oversampled_loader = DataLoader(dataloader.dataset, 
                                        batch_size=dataloader.batch_size, 
                                        sampler=sampler, 
                                        num_workers=dataloader.num_workers)
        
        return oversampled_loader
    
    
'''